## Import Needed Modules

In [11]:
pip install keras

In [10]:
import json
import random
import numpy as np

import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Data Analysis and Preprcessing

#### Read data

In [12]:
import json

# Ouvrir le fichier intents.json depuis le dossier de travail sur Colab
data_file = open('/content/intents.json').read()
intents = json.loads(data_file)


In [ ]:
data_file = open('/kaggle/input/books-dataset/intents.json').read()
intents = json.loads(data_file)

In [13]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']

### Preprocessing Text

In [14]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)

        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(words), "unique lemmatized words")
print (len(classes), "classes", classes)

161 documents
107 unique lemmatized words
51 classes ['Adventure stories', 'American fiction', 'Architecture', 'Art', 'Biography & Autobiography', 'Body, Mind & Spirit', 'Business & Economics', "Children's stories", 'Comics & Graphic Novels', 'Computers', 'Cooking', 'Detective and mystery stories', 'Drama', 'Education', 'English fiction', 'Family & Relationships', 'Fantasy fiction', 'Fiction', 'Foreign Language Study', 'Games', 'Health & Fitness', 'History', 'Humor', 'Juvenile Fiction', 'Juvenile Nonfiction', 'Language Arts & Disciplines', 'Law', 'Literary Collections', 'Literary Criticism', 'Medical', 'Music', 'Nature', 'Performing Arts', 'Philosophy', 'Photography', 'Poetry', 'Political Science', 'Psychology', 'Religion', 'Science', 'Science fiction', 'Self-Help', 'Social Science', 'Sports & Recreation', 'Travel', 'True Crime', 'Young Adult Fiction', 'book_search', 'goodbye', 'greeting', 'thanks']


### Initializing Training Data

In [20]:
# Préparer les données d'entraînement
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # Initialiser le sac de mots
    bag = []

    # Liste des mots tokenisés pour le pattern
    pattern_words = doc[0]

    # Lemmatiser chaque mot
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]

    # Créer le sac de mots
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # Création de la sortie
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# Mélanger les données
random.shuffle(training)

# Séparer les données en X (entrées) et Y (sorties) en utilisant une compréhension de liste
train_x = np.array([item[0] for item in training], dtype=np.float32)  # Sac de mots pour chaque document
train_y = np.array([item[1] for item in training], dtype=np.float32)  # Classe correspondante


print("Données d'entraînement créées")

Données d'entraînement créées


## Model Training

In [21]:
# Construire le modèle de réseau de neurones
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compiler le modèle avec le bon argument pour le taux d'apprentissage
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

print("Modèle compilé avec succès")


Modèle compilé avec succès


In [22]:
#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=500, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/500
33/33 ━━━━━━━━━━━━━━━━━━━━ 4s 45ms/step - accuracy: 0.0639 - loss: 3.9058
Epoch 2/500
33/33 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.0333 - loss: 3.8641
Epoch 3/500
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0379 - loss: 3.8541
Epoch 4/500
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0706 - loss: 3.7334
Epoch 5/500
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1424 - loss: 3.6858
Epoch 6/500
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1071 - loss: 3.6390
Epoch 7/500
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0653 - loss: 3.6416     
Epoch 8/500
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1966 - loss: 3.4346
Epoch 9/500
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2519 - loss: 3.2269
Epoch 10/500
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2903 - loss: 3.0704
Epoch 11/500
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3025 - loss: 2.9257 
Epoch 12/500
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - ac

model created


## Chatbot Prediction

### Function to clean user input

In [23]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]

    return sentence_words

### Function for Bag of Wrds

In [24]:
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)

    # bag of words - matrix of N words, vocabulary matrix
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:

                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

### Function for Class Prediction

In [25]:
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})

    return return_list

### Function to get chatbot response

In [26]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break

    return result

### Chatbot Function

In [27]:
def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

## Chatbot

In [28]:
chatbot_response('Recommend a book in History')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 233ms/step


{'Book': 'Tyranny of the Majority',
 'Feedback': "At last...the public hearing she was denied...These essays reveal keen powers of analysis applied to some of the most obdurate problems that bedevil electoral politics. Anyone who cares about the mechanisms of democracy should be engaged by her tough-minded explorations. It doesn't matter where you think you stand: it's all here, to argue or agree with. -- Henry Louis Gates, Jr. Lani Guinier's fascinating book is a prophetic intervention into a public conversation we desperately need to rejuvenate. There is no doubt that her powerful voice will produce good consequences for our nation and world. -- Cornel West, Author of Race Matters Intriguing and desperately needed... -- The San Francisco Chronicle",
 'Rate': 3.82}